# Automatically detect and model the background emission using LUCI
In this notebook, we will demonstrate the tools in LUCI to detect and model the background.

To do this, we apply the following steps:
1) Use a segmentation algorithm to find the background pixels
2) Use PCA to construct a subspace representing the background component
    - This includes the sky lines
3) Project each background pixel into a truncated PCA space
4) Interpolate masked pixels into PCA space
    - This can be done by either standard interpolation or using an artificial neural network

All of these steps have been wrapped into a single LUCI call for convenience. Implementation details can be found in our article.

In [28]:
import os
import sys
import matplotlib.pyplot as plt
plt.style.use('/home/carterrhea/Documents/LUCI/light.mplstyle')

In [29]:
# Define user-specific variables
n_components = 50  # Number of PCA components to calculate
n_components_keep = 2  # Number of PCA components to keep
outputDir = '/home/carterrhea/Documents/NGC4449'  # Output directory for plots and data
Luci_path = '/home/carterrhea/Documents/LUCI/'
cube_dir = '/home/carterrhea/Documents/NGC4449'  # Path to data cube
cube_name = 'NGC4449_SN3'  # don't add .hdf5 extension
object_name = 'NGC4449'
redshift = 0.0004  # Redshift
resolution = 5000

path = os.path.abspath(Luci_path)
sys.path.insert(0, path)  # add LUCI to the available paths
plt.style.use(os.path.join(Luci_path, 'light.mplstyle'))
from LuciBase import Luci

In [ ]:
cube = Luci(Luci_path, cube_dir+'/'+cube_name, cube_dir, object_name, redshift, resolution, mdn=False)


Reading in data...


 22%|██▏       | 2/9 [00:31<01:57, 16.84s/it]

In [ ]:
BkgTransformedPCA, pca, interpolatedSourcePixels = cube.create_background_subspace(x_min=500, x_max=1500, y_min=500, y_max=1500, n_components=n_components, n_components_keep=n_components_keep)

In [ ]:
import numpy as np

In [ ]:
print(np.sum([pca.components_[i] * BkgTransformedPCA[0,i] for i in range(n_components_keep)], axis=0))

In [ ]:
max_spectral = np.argmin(np.abs([1e7 / wavelength - 630 for wavelength in cube.spectrum_axis]))
min_spectral = np.argmin(np.abs([1e7 / wavelength - 680 for wavelength in cube.spectrum_axis]))
plt.plot(1e7/cube.spectrum_axis[min_spectral: max_spectral], pca.mean_-np.sum([pca.components_[i] * BkgTransformedPCA[0,i] for i in range(n_components_keep)], axis=0))
plt.plot(1e7/cube.spectrum_axis[min_spectral: max_spectral], pca.mean_, linewidth=3, linestyle='-.', label='Mean')
plt.plot(1e7/cube.spectrum_axis[min_spectral: max_spectral],
         pca.mean_ - np.sum([pca.components_[i] * interpolatedSourcePixels[10][i] for i in range(n_components_keep)], axis=0),
         linewidth=3, linestyle='--', label='Reconstructed Source 1')